In [1]:
def preprocess(X_train):
    X_train = X_train[X_train.columns[2:]]
    X_train = X_train.fillna(X_train.mean())    
    X_train["수술연월일"] = pd.to_datetime(X_train["수술연월일"])
    X_train["year"] = X_train["수술연월일"].dt.year
    X_train["month"] = X_train["수술연월일"].dt.month
    X_train["day"] = X_train["수술연월일"].dt.day
    X_train["day_name"] = X_train["수술연월일"].dt.day_name()

    date_list=[]
    for i in range(len(X_train)):
        date_list.append(str(X_train["year"][i]).zfill(2)+str(X_train["month"][i]).zfill(2) +str(X_train["day"][i]).zfill(2))

    X_train["수술연월일"] = date_list

    encoder = LabelEncoder()
    X_train["day_name"] = pd.DataFrame(encoder.fit_transform(np.reshape(np.array(X_train["day_name"]), (-1,1))))

    index = [i for i in range(28)if i != 24]
    X_train = X_train[X_train.columns[index]]
    return X_train

# Import data& libraries 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import itertools

In [3]:
path= "/mnt/d/Data/"

X_train = pd.read_csv(os.path.join(path, "train.csv"))
X_train = X_train.sample(frac=1)
y = X_train["N_category"]
X_train = X_train[X_train.columns[:-1]]
X_train = X_train[X_train.columns[3:]]    #ID, 이미지 경로 제거
X_test = pd.read_csv(os.path.join(path, "test.csv"))
submission = pd.read_csv(os.path.join(path, "sample_submission.csv"))

# Fill missing values

In [4]:
X_train = X_train.fillna(X_train.mean())

/tmp/ipykernel_930/3112579809.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  X_train = X_train.fillna(X_train.mean())


# Process categorical values

In [5]:
X_train["수술연월일"] = pd.to_datetime(X_train["수술연월일"])

X_train["year"] = X_train["수술연월일"].dt.year
X_train["month"] = X_train["수술연월일"].dt.month
X_train["day"] = X_train["수술연월일"].dt.day
X_train["day_name"] = X_train["수술연월일"].dt.day_name()
date_list=[]
for i in range(len(X_train)):
    date_list.append(str(X_train["year"][i]).zfill(2)+str(X_train["month"][i]).zfill(2) +str(X_train["day"][i]).zfill(2))
X_train["수술연월일"] = date_list

encoder = LabelEncoder()
X_train["day_name"] = pd.DataFrame(encoder.fit_transform(np.reshape(np.array(X_train["day_name"]), (-1,1))))

index = [i for i in range(28)if i != 24]
X_train = X_train[X_train.columns[index]]

/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y, stratify=y, test_size=0.2)

# Train vanilla RF 

In [8]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [10]:
model.score(X_valid, y_valid)

0.755

In [11]:
pred = model.predict(X_valid)
f1_score(pred, y_valid)

0.7782805429864253

# Check column combinations 

In [12]:
len(X_train.columns)

27

In [13]:
column_list = X_train.columns

In [48]:
score_list = []
columns = []
for i in range(25, 28, 1):
    print(i)
    columns_combi = list(itertools.combinations(column_list,i))
    for j in columns_combi:
        model = RandomForestClassifier()
        model.fit(X_train[list(j)], y_train)
        pred = model.predict(X_valid[list(j)])
        score = f1_score(pred, y_valid)
        score_list.append([score, j])

25
26
27


In [49]:
score_list.sort(reverse=True)

In [53]:
for i in score_list[:5]:
    print(i[0],len(i[1]))

0.8165137614678899 25
0.8073394495412843 25
0.8073394495412843 25
0.8073394495412843 25
0.8055555555555555 25


In [56]:
for i in score_list[:5]:
    print(i[1])

('나이', '수술연월일', '진단명', '암의 위치', '암의 개수', 'NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3', 'DCIS_or_LCIS_여부', 'DCIS_or_LCIS_type', 'T_category', 'ER', 'ER_Allred_score', 'PR', 'PR_Allred_score', 'HER2', 'HER2_IHC', 'HER2_SISH', 'HER2_SISH_ratio', 'BRCA_mutation', 'month', 'day', 'day_name')
('나이', '수술연월일', '진단명', '암의 위치', '암의 개수', '암의 장경', 'NG', 'HG_score_1', 'HG_score_2', 'HG_score_3', 'DCIS_or_LCIS_여부', 'DCIS_or_LCIS_type', 'T_category', 'ER', 'ER_Allred_score', 'PR', 'PR_Allred_score', 'KI-67_LI_percent', 'HER2', 'HER2_IHC', 'HER2_SISH', 'HER2_SISH_ratio', 'BRCA_mutation', 'month', 'day')
('나이', '수술연월일', '진단명', '암의 위치', '암의 개수', '암의 장경', 'NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3', 'DCIS_or_LCIS_여부', 'DCIS_or_LCIS_type', 'T_category', 'ER', 'ER_Allred_score', 'PR', 'PR_Allred_score', 'HER2', 'HER2_IHC', 'HER2_SISH', 'HER2_SISH_ratio', 'BRCA_mutation', 'month', 'day_name')
('나이', '수술연월일', '진단명', '암의 위치', '암의 개수', '암의 장경', 'NG', 'HG', 'HG_score_1', 'HG_score_2', 'DCIS_or_

In [ ]:
score_list = []
columns_combi = list(itertools.permutations(column_list,27))
print(len(columns_combi))a
for j in columns_combi:
    model = RandomForestClassifier()
    model.fit(X_train[j])
    pred = model.predict(X_valid)
    score = f1_score(pred, y_valid)
    score_list.append([j, score])